<a href="https://colab.research.google.com/github/prahladn/music_1/blob/master/Attention_LSTM_network_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attention LSTM Network

In [ ]:
import numpy as np
import pandas as pd
import string
import tensorflow as tf
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import  Input
from keras.layers import LSTM
from keras.layers import Layer
import keras.backend as K
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence, text
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import json
import sklearn
from sklearn import preprocessing as skpp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Pre-processed song lyrics
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/original_cleaned_lyrics_100.csv')
data.head()

# English-only pre-processed song lyrics
# data = pd.read_csv('../dataset/english_cleaned_lyrics.csv')

,Unnamed: 0,index,song,genre,lyrics
0,0,0,ego-remix,Pop,Oh baby how you doing You know I'm gonna cut r...
1,1,1,then-tell-me,Pop,playin everything so easy it's like you seem s...
2,2,2,honesty,Pop,If you search For tenderness It isn't hard to ...
3,3,3,you-are-my-rock,Pop,Oh oh oh I oh oh oh I If I wrote a book about ...
4,4,4,black-culture,Pop,Party the people the people the party it's pop...


In [ ]:
numpy_data = data['lyrics'].values
max_words = 30000

# Create a new Tokenizer
tokenizer = text.Tokenizer(num_words=max_words, oov_token='<UNK>')
# Feed our song lyrics to the Tokenizer
tokenizer.fit_on_texts(numpy_data)

# Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index

with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)
    
tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= max_words} # because tokenizer is 1 indexed
tokenizer.word_index[tokenizer.oov_token] = max_words + 1
indexed_data = tokenizer.texts_to_sequences(numpy_data)
indexed_data = np.array(indexed_data)

label_encoder = skpp.LabelEncoder()
indexed_labels = np.array(label_encoder.fit_transform(data['genre'].values))
# label_encoder.inverse_transform(np.array([10, 8])) # To get original genre text back

num_test = 30000

#Shuffling
random_indexes = np.random.permutation(len(indexed_labels))
indexed_data = indexed_data[random_indexes]
indexed_labels = indexed_labels[random_indexes]

X_train = indexed_data[:-num_test]
y_train = indexed_labels[:-num_test]
X_test  = indexed_data[-num_test:]
y_test  = indexed_labels[-num_test:]

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

num_words = max_words + 2
# Truncate and pad input sequences
max_review_length = 1000

X_train_padded = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test_padded = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [ ]:

print(X_train_padded.shape)
print(X_test_padded.shape)

(197449, 1000)
(30000, 1000)


In [ ]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
# Hyperparameters
embedding_vector_length = 100

#LSTM model 
model = Sequential()
model.add(Embedding(num_words, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(60, return_sequences=True))
model.add(attention())

model.add(Dense(11, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


#early stopping
early_stopping = EarlyStopping(monitor = 'accuracy', verbose = 1, patience = 5)

print(model.summary())


model.fit(X_train_padded, y_train, epochs=3, validation_data = (X_test_padded, y_test), batch_size=64, callbacks = [early_stopping])

# Final evaluation of the model on the test set
scores = model.evaluate(X_test_padded, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[-1]*100))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 100)         3000200   
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000, 60)          38640     
_________________________________________________________________
attention_1 (attention)      (None, 60)                1060      
_________________________________________________________________
dense_1 (Dense)              (None, 11)                671       
Total params: 3,040,571
Trainable params: 3,040,571
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
3086/3086 [==============================] - 233s 75ms/step - loss: 1.4475 - accuracy: 0.5402 - val_loss: 1.3137 - val_accuracy: 0.5788
Epoch 2/3
3086/3086 [==============================] - 232s 75ms/step - loss: 1.2570 - accu

In [ ]:
# Save model architecture and weights for later use
model.save('lstm_attempt.h5')